In [ ]:
pip install yfinance tensorflow  sklearn pandas


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import datetime


In [ ]:

stock_symbol = 'nhpc.ns'
start_date = '2010-01-01'
end_date = datetime.datetime.today().strftime('%Y-%m-%d')

data = yf.download(stock_symbol, start=start_date, end=end_date)
data = data[['Close']]

print(data.tail())


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price           Close
Ticker        NHPC.NS
Date                 
2025-02-18  73.010002
2025-02-19  74.620003
2025-02-20  79.800003
2025-02-21  79.889999
2025-02-24  77.239998


In [ ]:

news_data = [
    ("Apple announces new iPhone release", "positive"),
    ("AAPL stock declines due to market downturn", "negative"),
    ("Apple reports record earnings", "positive"),
    ("AAPL faces lawsuit over patent infringement", "negative"),
    ("Apple partners with Tesla for AI advancements", "positive")
]

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

news_sentiment = []
for news_item in news_data:
    sentiment_score = sia.polarity_scores(news_item[0])['compound']
    news_sentiment.append([news_item[0], sentiment_score])

news_df = pd.DataFrame(news_sentiment, columns=['headline', 'sentiment'])
print(news_df)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


                                        headline  sentiment
0             Apple announces new iPhone release     0.0000
1     AAPL stock declines due to market downturn     0.0000
2                  Apple reports record earnings     0.0000
3    AAPL faces lawsuit over patent infringement    -0.6124
4  Apple partners with Tesla for AI advancements     0.0000


In [ ]:

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Close']])


sequence_length = 3000

X, y = [], []
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i, 0])
    y.append(scaled_data[i, 0])

X = np.array(X)
y = np.array(y)

X = np.reshape(X, (X.shape[0], X.shape[1], 1))

print(X.shape, y.shape)


(740, 3000, 1) (740,)


In [ ]:

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.save('stock_pre_model.keras')

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
model_checkpoint = ModelCheckpoint('/content/stock_pre_model.keras', save_best_only=True, monitor='val_loss', verbose=1)

callbacks = [early_stop, reduce_lr, model_checkpoint]

history = model.fit(
    X, y,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.1376
Epoch 1: val_loss improved from inf to 0.02234, saving model to /content/stock_pre_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - loss: 0.1346 - val_loss: 0.0223 - learning_rate: 0.0010
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0098
Epoch 2: val_loss improved from 0.02234 to 0.00161, saving model to /content/stock_pre_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 85s 4s/step - loss: 0.0097 - val_loss: 0.0016 - learning_rate: 0.0010
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0031
Epoch 3: val_loss did not improve from 0.00161
19/19 ━━━━━━━━━━━━━━━━━━━━ 79s 4s/step - loss: 0.0031 - val_loss: 0.0031 - learning_rate: 0.0010
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0024
Epoch 4: val_loss did not improve from 0.00161
19/19 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - loss: 0.0024 - val_loss: 0.0022 - learning_rate: 0.0010
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0022
Epoc

In [1]:

last_sequence = scaled_data[-sequence_length:]
last_sequence = np.reshape(last_sequence, (1, sequence_length, 1))
predicted_price = model.predict(last_sequence)

predicted_price = scaler.inverse_transform(predicted_price)
print(f"Predicted next day's stock price: {predicted_price[0][0]:.2f}")


NameError: name 'scaled_data' is not defined